# Yahoo Finance Balance Sheet Data
### by Antonio Vargas

The idea is to automate data gathering for balance-sheet data from Yahoo Finance. 

## Introduction

This used to be easy, as Yahoo Finance used to have user-friendly export-to-Excel options (I used this in college!), but according to multiple forums ([Quora forum: Did Yahoo Stock API shut down?](https://www.quora.com/Did-Yahoo-Stock-API-shut-down)), in budget-cutting efforts after Verizon acquisitions, Yahoo shutdown much of their export options (easy Excel export and API), so third-party webscraping tools are needed. 

I did find this tool that automates the webscraping (based on webscraping tool BeautifulSoup) from Yahoo Finance: 

___

https://pypi.org/project/yahoofinancials/

A python module that returns stock, cryptocurrency, forex, mutual fund, commodity futures, ETF, and US Treasury financial data from Yahoo Finance.

___

It apparently accepts a single ticker or an array of tickers as an input. 

In my testing, I found that there weren't any significant performance gains in setting an array as an input in place of looping through retrievals on single ticker inputs (it generally takes 6 seconds to retrieve balance-sheet JSON from each ticker). Thus, in the defined function, I do loop through all of the tickers as that allows for a success/fail statement to be printed at the end of each loop to indicate progress. I found this necessary as 6-seconds-per-ticker for many tickers can take awhile, thus it's nice to be able to track the progress of a long retrieval. 

## Importing yahoofinancials and other modules

I will begin by importing the yahoofinancials module and pandas (used to create DataFrame).

In [1]:
# importing modules
!python -m pip install yahoofinancials

import yahoofinancials
import pandas as pd
from yahoofinancials import YahooFinancials

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Defining a function and writing to a file

The returned JSON object from the `get_financials_stmts()` function is basically in the structure "balanceSheetHistory" > stock-ticker > date > balance-sheet fields. 

The logic for the function is to 

1. loop through all inputted tickers
2. for each ticker, create a dictionary hold the ticker value, date, and balance-sheet values
3. append each dictionary to a list of dictionaries
4. convert the list of dictionaries to a DataFrame
5. re-order the columns so that the stock-ticker and date fields come first
6. return the DataFrame 

Additionally, I've specified the function to output print statements for each row to indicate whether the retrieval was successful. I found this necessary as in testing, I found that attempting to retrieve approximately 250 stocks took about half an hour. For the long retrievals, I did appreciate the progress indicator. 

In [2]:
def from_tickers_to_dataframe(tickers):
    """This takes an array of tickers and returns a Dataframe with balance sheet information for those tickers"""
    list_dict = []
    for tick in tickers:
        ticker_yf = YahooFinancials(tick)
        ticker_bs = ticker_yf.get_financial_stmts('annual', 'balance', True)
        try:
            for date_bs in ticker_bs['balanceSheetHistory'][tick]:
                ticker_dict = dict(stock_ticker = tick)
                date_dict = dict(date = list(date_bs.keys())[0])
                bs_dict = date_bs[list(date_bs.keys())[0]]
                row_dict = {**ticker_dict, **date_dict, **bs_dict}
                list_dict.append(row_dict)
                print('Stock {} ({}) retrieved successfully'.format(tick, list(date_bs.keys())[0]))
        except:
            print('Ticker value {} caused an error'.format(tick))
    sample_df = pd.DataFrame.from_dict(list_dict)
    columns_list = list(sample_df.columns)
    columns_list.remove('stock_ticker')
    columns_list.remove('date')
    columns_list = ['stock_ticker', 'date'] + columns_list
    sample_df = sample_df[columns_list]
    return sample_df

I will now use a list of tickers from a third-party site and attempt to retrieve the Balance Sheet information for all available stocks. 

In [3]:
## list of utility stocks: https://www.suredividend.com/utility-stocks-list/
tickers =\
['ABTZY',
'ABZPF',
'ABZPY',
'ACLLF',
'ACO.X.TO',
'ACO.Y.TO',
'AEE',
'AEMMY',
'AEP',
'AES',
'AGLNF',
'AGLXY',
'AGR',
'AILLI',
'AILLL',
'ALE',
'ALPVN',
'APAJF',
'APRCP',
'APRDM',
'APRDN',
'APRDO',
'APRDP',
'APTL',
'APU',
'AQN',
'AQN.TO',
'ARTNA',
'ARTNB',
'ATO',
'AVA',
'AWK',
'AWR',
'AY',
'BEP',
'BEP.UN.TO',
'BIP',
'BIP.UN.TO',
'BKH',
'BLX.TO',
'BRLXF',
'CDUAF',
'CESDY',
'CGHOF',
'CIG',
'CLPHF',
'CLPHY',
'CLPXF',
'CMS',
'CNIG',
'CNLHO',
'CNLTN',
'CNP',
'CNTHP',
'CPK',
'CPL',
'CPX.TO',
'CPXWF',
'CPYYF',
'CPYYY',
'CRPJY',
'CSQSY',
'CTPTY',
'CTPZY',
'CTWS',
'CU.TO',
'CU.X.TO',
'CUP.U.TO',
'CUPUF',
'CWAFF',
'CWCO',
'CWT',
'CZZ',
'D',
'DEIPY',
'DRXGY',
'DTE',
'DUK',
'ECIFY',
'ED',
'EDPFY',
'EDRWY',
'EE',
'EGDCY',
'EGIEY',
'EIX',
'ELEZF',
'ELEZY',
'ELP',
'EMA.TO',
'EMRAF',
'ENAKF',
'ENGGF',
'ENGGY',
'ENGIY',
'ENGQF',
'ENIA',
'ENIC',
'ENLAY',
'EOCC',
'EONGY',
'EPWDF',
'EQUEY',
'ES',
'ESOCF',
'ETR',
'EVNVY',
'EVRG',
'EXC',
'EYUBY',
'FE',
'FOJCF',
'FOJCY',
'FPHHF',
'FSGCY',
'FTS',
'FTS.TO',
'GASNF',
'GASNY',
'GGDVF',
'GGDVY',
'GWEFF',
'GWR.TO',
'GWRS',
'H.TO',
'HE',
'HGKGF',
'HGKGY',
'HNP',
'HOKCF',
'HOKCY',
'HPIFF',
'HUNGF',
'HYFXF',
'IBDRY',
'IBDSF',
'IDA',
'IENVF',
'IESFY',
'INE.TO',
'INGXF',
'IRDEF',
'IRDEY',
'JE',
'JE.TO',
'KAEPF',
'KWH.UN.TO',
'LNT',
'MGEE',
'MPCIY',
'MPRWL',
'MSEX',
'MWTCF',
'MWTCY',
'NEE',
'NEP',
'NESW',
'NGG',
'NGGTF',
'NI',
'NJR',
'NPI.TO',
'NPIFF',
'NRG',
'NWE',
'NWN',
'NYLD',
'NYLD.A',
'OEZVF',
'OEZVY',
'OGE',
'OGS',
'ORA',
'OSGSF',
'OTTR',
'PCG',
'PEG',
'PEGI',
'PEGI.TO',
'PEGRF',
'PEGRY',
'PIF.TO',
'PNM',
'PNW',
'POR',
'PPAAF',
'PPAAY',
'PPL',
'RAMPF',
'RBSFY',
'RDEIY',
'RGCO',
'RNW.TO',
'RSHYY',
'RUBSF',
'SAUNF',
'SCG',
'SFDPF',
'SJI',
'SJW',
'SNMRF',
'SNMRY',
'SO',
'SPB.TO',
'SPH',
'SPKE',
'SR',
'SRE',
'SSEZF',
'SSEZY',
'STRNY',
'SUUIF',
'SVTRF',
'SWX',
'SZEVF',
'SZEVY',
'TA.TO',
'TAC',
'TERRF',
'TEZNY',
'TKGSF',
'TNABF',
'TNABY',
'TORW',
'TTAPF',
'TTAPY',
'TTWSF',
'UGI',
'UTL',
'UUGRY',
'UUGWF',
'VETTF',
'VNR.TO',
'VNRCF',
'VVC',
'WEC',
'WTR',
'XEL',
'XNGSF',
'XNGSY',
'YORW']

In [4]:
%%time
## from testing, at (18sec/3stocks) = 6-seconds-per-stock rate, I expect this to take about a half-hour
test_utility_df = from_tickers_to_dataframe(tickers)

Stock ABTZY (2018-12-31) retrieved successfully
Stock ABTZY (2017-12-31) retrieved successfully
Stock ABTZY (2017-01-01) retrieved successfully
Stock ABTZY (2015-12-31) retrieved successfully
Stock ABZPF (2018-12-31) retrieved successfully
Stock ABZPF (2017-12-31) retrieved successfully
Stock ABZPF (2016-12-31) retrieved successfully
Stock ABZPF (2015-12-31) retrieved successfully
Ticker value ABZPY caused an error
Stock ACLLF (2018-12-31) retrieved successfully
Stock ACLLF (2017-12-31) retrieved successfully
Stock ACLLF (2016-12-31) retrieved successfully
Stock ACLLF (2015-12-31) retrieved successfully
Stock AEE (2018-12-31) retrieved successfully
Stock AEE (2017-12-31) retrieved successfully
Stock AEE (2016-12-31) retrieved successfully
Stock AEE (2015-12-31) retrieved successfully
Stock AEMMY (2018-12-31) retrieved successfully
Stock AEMMY (2017-12-31) retrieved successfully
Stock AEMMY (2016-12-31) retrieved successfully
Stock AEMMY (2015-12-31) retrieved successfully
Stock AEP (20

Stock CPL (2018-12-31) retrieved successfully
Stock CPL (2017-12-31) retrieved successfully
Stock CPL (2016-12-31) retrieved successfully
Stock CPL (2015-12-31) retrieved successfully
Stock CPX.TO (2018-12-31) retrieved successfully
Stock CPX.TO (2017-12-31) retrieved successfully
Stock CPX.TO (2016-12-31) retrieved successfully
Stock CPX.TO (2015-12-31) retrieved successfully
Stock CPXWF (2018-12-31) retrieved successfully
Stock CPXWF (2017-12-31) retrieved successfully
Stock CPXWF (2016-12-31) retrieved successfully
Stock CPXWF (2015-12-31) retrieved successfully
Stock CPYYF (2018-12-31) retrieved successfully
Stock CPYYF (2017-12-31) retrieved successfully
Stock CPYYF (2016-12-31) retrieved successfully
Stock CPYYF (2015-12-31) retrieved successfully
Stock CPYYY (2018-12-31) retrieved successfully
Stock CPYYY (2017-12-31) retrieved successfully
Stock CPYYY (2016-12-31) retrieved successfully
Stock CPYYY (2015-12-31) retrieved successfully
Stock CRPJY (2018-12-31) retrieved successfu

Stock ETR (2018-12-31) retrieved successfully
Stock ETR (2017-12-31) retrieved successfully
Stock ETR (2016-12-31) retrieved successfully
Stock ETR (2015-12-31) retrieved successfully
Stock EVNVY (2018-09-30) retrieved successfully
Stock EVNVY (2017-09-30) retrieved successfully
Stock EVNVY (2016-09-30) retrieved successfully
Stock EVNVY (2015-09-30) retrieved successfully
Stock EVRG (2018-12-31) retrieved successfully
Stock EVRG (2017-12-31) retrieved successfully
Stock EVRG (2016-12-31) retrieved successfully
Stock EVRG (2015-12-31) retrieved successfully
Stock EXC (2018-12-31) retrieved successfully
Stock EXC (2017-12-31) retrieved successfully
Stock EXC (2016-12-31) retrieved successfully
Stock EXC (2015-12-31) retrieved successfully
Ticker value EYUBY caused an error
Stock FE (2018-12-31) retrieved successfully
Stock FE (2017-12-31) retrieved successfully
Stock FE (2016-12-31) retrieved successfully
Stock FE (2015-12-31) retrieved successfully
Stock FOJCF (2018-12-31) retrieved su

Ticker value MWTCY caused an error
Stock NEE (2018-12-31) retrieved successfully
Stock NEE (2017-12-31) retrieved successfully
Stock NEE (2016-12-31) retrieved successfully
Stock NEE (2015-12-31) retrieved successfully
Stock NEP (2018-12-31) retrieved successfully
Stock NEP (2017-12-31) retrieved successfully
Stock NEP (2016-12-31) retrieved successfully
Stock NEP (2015-12-31) retrieved successfully
Stock NESW (2018-12-31) retrieved successfully
Stock NESW (2017-12-31) retrieved successfully
Stock NESW (2016-12-31) retrieved successfully
Stock NESW (2015-12-31) retrieved successfully
Stock NGG (2018-03-31) retrieved successfully
Stock NGG (2017-03-31) retrieved successfully
Stock NGG (2016-03-31) retrieved successfully
Stock NGG (2015-03-31) retrieved successfully
Stock NGGTF (2018-03-31) retrieved successfully
Stock NGGTF (2017-03-31) retrieved successfully
Stock NGGTF (2016-03-31) retrieved successfully
Stock NGGTF (2015-03-31) retrieved successfully
Stock NI (2018-12-31) retrieved s

Stock SNMRF (2018-12-31) retrieved successfully
Stock SNMRF (2017-12-31) retrieved successfully
Stock SNMRF (2016-12-31) retrieved successfully
Stock SNMRF (2015-12-31) retrieved successfully
Stock SNMRY (2018-12-31) retrieved successfully
Stock SNMRY (2017-12-31) retrieved successfully
Stock SNMRY (2016-12-31) retrieved successfully
Stock SNMRY (2015-12-31) retrieved successfully
Stock SO (2018-12-31) retrieved successfully
Stock SO (2017-12-31) retrieved successfully
Stock SO (2016-12-31) retrieved successfully
Stock SO (2015-12-31) retrieved successfully
Stock SPB.TO (2018-12-31) retrieved successfully
Stock SPB.TO (2017-12-31) retrieved successfully
Stock SPB.TO (2016-12-31) retrieved successfully
Stock SPB.TO (2015-12-31) retrieved successfully
Stock SPH (2018-09-29) retrieved successfully
Stock SPH (2017-09-30) retrieved successfully
Stock SPH (2016-09-24) retrieved successfully
Stock SPH (2015-09-26) retrieved successfully
Stock SPKE (2018-12-31) retrieved successfully
Stock SPK

As expected, it took about a half-hour to retrieve the balance-sheet data and write it to a DataFrame.

I will now preview the DataFrame to ensure it looks as intended.

In [5]:
test_utility_df

,stock_ticker,date,accountsPayable,capitalSurplus,cash,commonStock,deferredLongTermAssetCharges,deferredLongTermLiab,goodWill,intangibleAssets,...,propertyPlantEquipment,retainedEarnings,shortLongTermDebt,shortTermInvestments,totalAssets,totalCurrentAssets,totalCurrentLiabilities,totalLiab,totalStockholderEquity,treasuryStock
0,ABTZY,2018-12-31,1.593315e+10,1.301320e+10,5.903303e+10,5.694600e+09,2.324773e+09,5.830950e+08,5.626191e+10,6.832545e+09,...,2.214308e+11,1.527419e+11,1.070297e+10,NaN,554588234000,136008355000,77275227000,337324703000,174705140000,3.255433e+09
1,ABTZY,2017-12-31,1.062728e+10,1.301320e+10,6.487021e+10,5.694600e+09,1.525630e+09,3.954190e+08,4.130869e+10,6.002482e+09,...,2.132325e+11,1.369101e+11,2.072233e+10,NaN,491931697000,117877171000,73067285000,299660769000,154698419000,-9.195230e+08
2,ABTZY,2017-01-01,2.221091e+10,1.301320e+10,6.385753e+10,5.694600e+09,1.893878e+09,4.114990e+08,4.124963e+10,3.222123e+09,...,2.022376e+11,1.227944e+11,7.698261e+09,NaN,465994925000,109104207000,41989346000,292332301000,139962243000,-1.539948e+09
3,ABTZY,2015-12-31,9.413256e+09,7.683568e+09,6.358188e+10,5.694600e+09,6.995490e+08,2.940710e+08,2.073972e+09,6.271652e+09,...,1.439977e+11,1.065212e+11,3.133346e+09,NaN,340117863000,97033419000,34163210000,193133421000,118219958000,-1.679452e+09
4,ABZPF,2018-12-31,8.999633e+09,1.258889e+10,4.634304e+10,7.358604e+09,2.233695e+09,3.884680e+08,4.022441e+10,5.552615e+09,...,2.071104e+11,1.084877e+11,8.697404e+09,NaN,389662176000,88708607000,46815020000,253085509000,127712916000,-7.223200e+08
5,ABZPF,2017-12-31,7.813066e+09,1.258889e+10,3.569963e+10,7.358604e+09,1.406796e+09,2.634360e+08,4.027034e+10,5.810878e+09,...,2.040253e+11,9.706631e+10,2.069275e+10,NaN,361476999000,67961596000,49312291000,237498535000,115396370000,-1.617436e+09
6,ABZPF,2016-12-31,7.591617e+09,1.258889e+10,4.709474e+10,7.358604e+09,1.802570e+09,2.737250e+08,4.027034e+10,6.173988e+09,...,1.929764e+11,8.665757e+10,7.458363e+09,NaN,357006894000,73649187000,32802506000,244797674000,105114419000,-1.490647e+09
7,ABZPF,2015-12-31,5.749973e+09,1.258889e+10,5.109827e+10,7.358604e+09,5.848790e+08,2.698890e+08,1.094687e+09,6.228144e+09,...,1.348106e+11,7.887027e+10,2.368161e+09,NaN,242489246000,70409021000,22553200000,140875679000,97568521000,-1.249246e+09
8,ACLLF,2018-12-31,9.210000e+08,1.100000e+07,6.910000e+08,1.690000e+08,8.500000e+07,NaN,8.200000e+07,6.720000e+08,...,1.786500e+10,3.535000e+09,5.080000e+08,NaN,23344000000,2097000000,1748000000,15902000000,3755000000,4.000000e+07
9,ACLLF,2017-12-31,8.940000e+08,1.000000e+07,5.010000e+08,1.670000e+08,8.700000e+07,NaN,7.100000e+07,5.870000e+08,...,1.734300e+10,3.352000e+09,2.000000e+07,NaN,21786000000,2268000000,1037000000,14683000000,3527000000,-2.000000e+06


The DataFrame looks about right. I will now write it to a csv file. 

In [6]:
test_utility_df.to_csv('test_utilities_balancesheetdata.csv', index = False)